## Victorian Road Crash Statistics and Analysis

Road statistics are well reported, but are there trends and relationships around preventative measures such as speed limit restrictions, road upgrades and road safety cameras.  We will be asking which one is the most effective at preventing incidents and what type of incidents.  

We will be diving into the data to see if our hypothesis that road upgrades are likely to be the most effective.

Key questions to be considered are:
* Is there a correlation between the number and type of road incidents and the speed limit?  Eg - Fatalities / Serious injury / Minor or no injury as classified by VicRoads.
* Does local government spending improve road conditions to prevent incidents? 
* Does the overall demographics (age / average income / gender) of local government areas have an impact on the number of incidents?
* Do speed camera locations actually decrease crash numbers within a radius of 1km from the incidents v those outside 1 km of the speed camera?


In [1]:
#Import dependencies
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
from scipy import stats

pd.set_option('display.max_rows', 80)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)


In [2]:
#Import the inital CSV
crash_data = pd.read_csv("Resources/Crash_Statistics_Victoria_clean.csv")
crash_data.head()

,OBJECTID,ACCIDENT_DATE,ACCIDENT_TIME,ALCOHOLTIME,ACCIDENT_TYPE,DAY_OF_WEEK,LIGHT_CONDITION,ROAD_GEOMETRY,SEVERITY,SPEED_ZONE,NODE_ID,LONGITUDE,LATITUDE,NODE_TYPE,LGA_NAME,REGION_NAME,VICGRID_X,VICGRID_Y,TOTAL_PERSONS,INJ_OR_FATAL,FATALITY,SERIOUSINJURY,OTHERINJURY,NONINJURED,MALES,FEMALES,BICYCLIST,PASSENGER,DRIVER,PEDESTRIAN,PILLION,MOTORIST,UNKNOWN,PED_CYCLIST_5_12,PED_CYCLIST_13_18,OLD_PEDESTRIAN,OLD_DRIVER,YOUNG_DRIVER,ALCOHOL_RELATED,UNLICENCSED,NO_OF_VEHICLES,HEAVYVEHICLE,PASSENGERVEHICLE,MOTORCYCLE,PUBLICVEHICLE,RMA,STAT_DIV_NAME
0,3401744,1/07/2013,18.30.00,Yes,Struck Pedestrian,Monday,Dark Street lights on,Not at intersection,Serious injury accident,60 km/hr,261534,144.96980,-37.82202,Non-Intersection,MELBOURNE,METROPOLITAN NORTH WEST REGION,2497341.176,2408778.118,3,1,0,1,0,2,2,1,0,1,1,1,0,0,0,0,0,0,0,0,No,0,1,0,1,0,0,Local Road,Metro
1,3401745,2/07/2013,16.40.00,No,Collision with vehicle,Tuesday,Day,T intersection,Serious injury accident,60 km/hr,66573,145.14671,-37.83166,Intersection,WHITEHORSE,METROPOLITAN SOUTH EAST REGION,2512914.561,2407698.334,2,1,0,1,0,1,1,1,0,0,1,1,0,0,0,0,0,1,1,0,No,0,3,0,3,0,0,Arterial Other,Metro
2,3401746,2/07/2013,13.15.00,No,Collision with a fixed object,Tuesday,Day,T intersection,Serious injury accident,50 km/hr,261533,144.80134,-37.74003,Intersection,BRIMBANK,METROPOLITAN NORTH WEST REGION,2482490.641,2417859.686,1,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,Yes,0,1,0,1,0,0,Local Road,Metro
3,3401747,2/07/2013,16.45.00,No,Collision with a fixed object,Tuesday,Day,Not at intersection,Serious injury accident,110 km/hr,263495,145.07011,-37.17891,Non-Intersection,MITCHELL,NORTHERN REGION,2506225.129,2480145.341,3,2,0,2,0,1,1,2,0,2,1,0,0,0,0,0,0,0,0,0,No,0,1,0,1,0,0,Freeway,Country
4,3401748,2/07/2013,15.48.00,No,Collision with vehicle,Tuesday,Day,Not at intersection,Other injury accident,50 km/hr,261664,144.96530,-37.81808,Non-Intersection,MELBOURNE,METROPOLITAN NORTH WEST REGION,2496944.686,2409215.737,3,1,0,0,1,2,3,0,0,1,2,0,0,0,0,0,0,0,0,0,No,0,2,0,1,0,1,Local Road,Metro


In [3]:
#check dimensions
crash_data.shape

(65435, 47)

In [4]:
#check missing values
crash_data.isnull().sum()

OBJECTID                0
ACCIDENT_DATE           0
ACCIDENT_TIME           0
ALCOHOLTIME             0
ACCIDENT_TYPE           0
DAY_OF_WEEK          1146
LIGHT_CONDITION         0
ROAD_GEOMETRY           0
SEVERITY                0
SPEED_ZONE              0
NODE_ID                 0
LONGITUDE               0
LATITUDE                0
NODE_TYPE               6
LGA_NAME                0
REGION_NAME             0
VICGRID_X               0
VICGRID_Y               0
TOTAL_PERSONS           0
INJ_OR_FATAL            0
FATALITY                0
SERIOUSINJURY           0
OTHERINJURY             0
NONINJURED              0
MALES                   0
FEMALES                 0
BICYCLIST               0
PASSENGER               0
DRIVER                  0
PEDESTRIAN              0
PILLION                 0
MOTORIST                0
UNKNOWN                 0
PED_CYCLIST_5_12        0
PED_CYCLIST_13_18       0
OLD_PEDESTRIAN          0
OLD_DRIVER              0
YOUNG_DRIVER            0
ALCOHOL_RELA

In [5]:
#remove NA values based on row
crash_data = crash_data.dropna(axis = 0)

In [6]:
#check missing values
crash_data.isnull().sum()


OBJECTID             0
ACCIDENT_DATE        0
ACCIDENT_TIME        0
ALCOHOLTIME          0
ACCIDENT_TYPE        0
DAY_OF_WEEK          0
LIGHT_CONDITION      0
ROAD_GEOMETRY        0
SEVERITY             0
SPEED_ZONE           0
NODE_ID              0
LONGITUDE            0
LATITUDE             0
NODE_TYPE            0
LGA_NAME             0
REGION_NAME          0
VICGRID_X            0
VICGRID_Y            0
TOTAL_PERSONS        0
INJ_OR_FATAL         0
FATALITY             0
SERIOUSINJURY        0
OTHERINJURY          0
NONINJURED           0
MALES                0
FEMALES              0
BICYCLIST            0
PASSENGER            0
DRIVER               0
PEDESTRIAN           0
PILLION              0
MOTORIST             0
UNKNOWN              0
PED_CYCLIST_5_12     0
PED_CYCLIST_13_18    0
OLD_PEDESTRIAN       0
OLD_DRIVER           0
YOUNG_DRIVER         0
ALCOHOL_RELATED      0
UNLICENCSED          0
NO_OF_VEHICLES       0
HEAVYVEHICLE         0
PASSENGERVEHICLE     0
MOTORCYCLE 

In [7]:
#load local government expenditure data, skip first row
localgov = pd.read_csv("Resources/lge_cleanfile.csv", skiprows = 1)
localgov.head(10)

,LGA_NAME,Local Roads - Sealed-length,Local Roads - Unsealed - Formed & Sheeted-length,Local Roads - Unsealed - Natural Surface_length,Total_roadlength,Local Roads - Sealed-expenditure,Local Roads - Unsealed - Formed & Sheeted-expenditure,Local Roads - Unsealed - Natural Surface-expenditure,Roads Ancillary-expenditure,Total-expenditure
0,Alpine,412.0,233.0,215.0,860.0,3748129.0,265116.0,0.0,0.0,4013244.0
1,Ararat,764.0,1420.0,240.0,2424.0,7303459.0,2975868.0,2281.0,1361455.0,11845829.0
2,Ballarat,1044.0,351.0,40.0,1435.0,41874084.0,3196942.0,363289.0,1045194.0,46775611.0
3,Banyule,538.0,6.0,1.0,545.0,10329600.0,0.0,0.0,0.0,10329600.0
4,Bass Coast,563.0,373.0,7.0,943.0,7355926.0,1677983.0,0.0,789457.0,9923525.0
5,Baw Baw,1110.0,721.0,184.0,2015.0,13512590.0,3092182.0,0.0,1493686.0,19073493.0
6,Bayside,355.0,1.0,0.0,356.0,5547236.0,0.0,0.0,6045707.0,11592943.0
7,Benalla,560.0,713.0,78.0,1351.0,2504772.0,1474785.0,0.0,0.0,3979557.0
8,Boroondara,562.0,0.0,0.0,562.0,11436082.0,0.0,0.0,0.0,11436082.0
9,Brimbank,889.0,5.0,0.0,894.0,25715082.0,344474.0,0.0,1997635.0,28078056.0


In [8]:
#check null values
localgov.isnull().sum()

LGA_NAME                                                  0
Local Roads - Sealed-length                               0
Local Roads - Unsealed - Formed &  Sheeted-length         0
Local Roads - Unsealed - Natural Surface_length           0
Total_roadlength                                          0
Local Roads - Sealed-expenditure                          0
Local Roads - Unsealed - Formed &  Sheeted-expenditure    0
Local Roads - Unsealed - Natural Surface-expenditure      0
Roads Ancillary-expenditure                               0
Total-expenditure                                         0
dtype: int64

In [9]:
#unique lGA names on local gov dataset
localgov["LGA_NAME"].nunique()
localgov["LGA_NAME"].value_counts()

Maribyrnong              1
Colac Otway              1
Murrindindi              1
Ballarat                 1
Greater Bendigo          1
Corangamite              1
Northern Grampians       1
Greater Geelong          1
Glenelg                  1
Yarriambiack             1
Mornington Peninsula     1
Warrnambool              1
Ararat                   1
Benalla                  1
Maroondah                1
Moorabool                1
Stonnington              1
Golden Plains            1
Campaspe                 1
Whitehorse               1
Macedon Ranges           1
Towong                   1
Bass Coast               1
Southern Grampians       1
Mount Alexander          1
Whittlesea               1
Kingston                 1
Queenscliffe             1
Casey                    1
Surf Coast               1
Boroondara               1
Wangaratta               1
Central Goldfields       1
Nillumbik                1
Glen Eira                1
East Gippsland           1
Hume                     1
S

In [10]:
# Load in the csv data files
LGA_demo_path = "Resources/lga_demographics.csv"

# Read the data 
lga_demo_df = pd.read_csv(LGA_demo_path)

In [11]:
#Check the dataframe import
lga_demo_df

,LGA Name,LGA Code,Departmental Region,City_Regional,Index of Relative Socio-Economic Disadvantage (IRSD),Index of Relative Socio-Economic Disadvantage (IRSD) (rank),Percentage of highly disadvantaged SA1s,Percentage of highly disadvantaged SA1s (rank),People with income less than $400 per week,People with income less than $400 per week (rank),Median household income,Median household income (rank),Median house price,Median house price (rank),Journeys to work which are by car,Journeys to work which are by car (rank),People with at least 2 hour daily commute,People with at least 2 hour daily commute (rank),People at increased risk of alcohol-related harm on a single occasion of drinking,People at increased risk of alcohol-related harm on a single occasion of drinking (rank),People reporting fair or poor health status,People reporting fair or poor health status (rank),"Clients that received Alcohol & Drug Treatment Services per 1,000 population","Clients that received Alcohol & Drug Treatment Services per 1,000 population (rank)",Total_pop
0,ALPINE,20110,Hume,Regional,987,35,0.238,33,0.448,21,829,67,265000,50,0.439,77,0.083,35,0.561,6,0.193,10,1.0,78,12335
1,ARARAT,20260,Grampians,Regional,951,11,0.455,8,0.433,30,844,64,193000,68,0.687,38,0.000,0,0.482,31,0.144,45,11.5,4,11599
2,BALLARAT,20570,Grampians,Regional,981,29,0.315,23,0.417,39,988,44,294000,48,0.738,15,0.047,47,0.537,15,0.123,60,5.5,31,101689
3,BANYULE,20660,Northern Metropolitan,Metro,1047,67,0.083,60,0.369,68,1394,12,620000,17,0.641,55,0.206,5,0.407,64,0.167,31,3.7,54,121869
4,BASS COAST,20740,Gippsland,Regional,978,27,0.216,36,0.438,26,855,61,340000,38,0.714,25,0.068,40,0.452,44,0.177,20,7.1,17,32806
5,BAW BAW,20830,Gippsland,Regional,998,47,0.152,54,0.415,41,1025,40,309000,44,0.699,32,0.108,25,0.429,59,0.129,58,5.1,35,48477
6,BAYSIDE,20910,Southern Metropolitan,Metro,1091,77,0.023,73,0.327,76,1826,3,1250000,3,0.627,57,0.141,20,0.505,23,0.085,78,2.8,67,97092
7,BENALLA,21010,Hume,Regional,957,16,0.297,26,0.448,22,827,69,235500,59,0.693,34,0.000,0,0.402,66,0.098,71,3.2,62,13863
8,BENDIGO,22620,Loddon Mallee,Regional,983,31,0.280,28,0.413,43,991,43,320000,41,0.740,13,0.000,0,0.441,50,0.172,26,7.1,15,110479
9,BOROONDARA,21110,Eastern Metropolitan,Metro,1098,78,0.008,75,0.333,75,1893,1,1550000,1,0.566,68,0.055,43,0.488,28,0.103,68,2.5,70,167232


In [12]:
lga_demo_df.dtypes

LGA Name                                                                                     object
LGA Code                                                                                      int64
Departmental Region                                                                          object
City_Regional                                                                                object
Index of Relative Socio-Economic Disadvantage (IRSD)                                          int64
Index of Relative Socio-Economic Disadvantage (IRSD) (rank)                                   int64
Percentage of highly disadvantaged SA1s                                                     float64
Percentage of highly disadvantaged SA1s (rank)                                                int64
People with income less than $400 per week                                                  float64
People with income less than $400 per week (rank)                                             int64


In [13]:
#Rename the column name LGA Name to LGA
lga_demo_df2 = lga_demo_df.rename(columns={"LGA Name": "LGA"})


In [14]:
lga_demo_df2

,LGA,LGA Code,Departmental Region,City_Regional,Index of Relative Socio-Economic Disadvantage (IRSD),Index of Relative Socio-Economic Disadvantage (IRSD) (rank),Percentage of highly disadvantaged SA1s,Percentage of highly disadvantaged SA1s (rank),People with income less than $400 per week,People with income less than $400 per week (rank),Median household income,Median household income (rank),Median house price,Median house price (rank),Journeys to work which are by car,Journeys to work which are by car (rank),People with at least 2 hour daily commute,People with at least 2 hour daily commute (rank),People at increased risk of alcohol-related harm on a single occasion of drinking,People at increased risk of alcohol-related harm on a single occasion of drinking (rank),People reporting fair or poor health status,People reporting fair or poor health status (rank),"Clients that received Alcohol & Drug Treatment Services per 1,000 population","Clients that received Alcohol & Drug Treatment Services per 1,000 population (rank)",Total_pop
0,ALPINE,20110,Hume,Regional,987,35,0.238,33,0.448,21,829,67,265000,50,0.439,77,0.083,35,0.561,6,0.193,10,1.0,78,12335
1,ARARAT,20260,Grampians,Regional,951,11,0.455,8,0.433,30,844,64,193000,68,0.687,38,0.000,0,0.482,31,0.144,45,11.5,4,11599
2,BALLARAT,20570,Grampians,Regional,981,29,0.315,23,0.417,39,988,44,294000,48,0.738,15,0.047,47,0.537,15,0.123,60,5.5,31,101689
3,BANYULE,20660,Northern Metropolitan,Metro,1047,67,0.083,60,0.369,68,1394,12,620000,17,0.641,55,0.206,5,0.407,64,0.167,31,3.7,54,121869
4,BASS COAST,20740,Gippsland,Regional,978,27,0.216,36,0.438,26,855,61,340000,38,0.714,25,0.068,40,0.452,44,0.177,20,7.1,17,32806
5,BAW BAW,20830,Gippsland,Regional,998,47,0.152,54,0.415,41,1025,40,309000,44,0.699,32,0.108,25,0.429,59,0.129,58,5.1,35,48477
6,BAYSIDE,20910,Southern Metropolitan,Metro,1091,77,0.023,73,0.327,76,1826,3,1250000,3,0.627,57,0.141,20,0.505,23,0.085,78,2.8,67,97092
7,BENALLA,21010,Hume,Regional,957,16,0.297,26,0.448,22,827,69,235500,59,0.693,34,0.000,0,0.402,66,0.098,71,3.2,62,13863
8,BENDIGO,22620,Loddon Mallee,Regional,983,31,0.280,28,0.413,43,991,43,320000,41,0.740,13,0.000,0,0.441,50,0.172,26,7.1,15,110479
9,BOROONDARA,21110,Eastern Metropolitan,Metro,1098,78,0.008,75,0.333,75,1893,1,1550000,1,0.566,68,0.055,43,0.488,28,0.103,68,2.5,70,167232


In [15]:
crash_demographics = crash_data.copy()

In [16]:
crash_demographics

,OBJECTID,ACCIDENT_DATE,ACCIDENT_TIME,ALCOHOLTIME,ACCIDENT_TYPE,DAY_OF_WEEK,LIGHT_CONDITION,ROAD_GEOMETRY,SEVERITY,SPEED_ZONE,NODE_ID,LONGITUDE,LATITUDE,NODE_TYPE,LGA_NAME,REGION_NAME,VICGRID_X,VICGRID_Y,TOTAL_PERSONS,INJ_OR_FATAL,FATALITY,SERIOUSINJURY,OTHERINJURY,NONINJURED,MALES,FEMALES,BICYCLIST,PASSENGER,DRIVER,PEDESTRIAN,PILLION,MOTORIST,UNKNOWN,PED_CYCLIST_5_12,PED_CYCLIST_13_18,OLD_PEDESTRIAN,OLD_DRIVER,YOUNG_DRIVER,ALCOHOL_RELATED,UNLICENCSED,NO_OF_VEHICLES,HEAVYVEHICLE,PASSENGERVEHICLE,MOTORCYCLE,PUBLICVEHICLE,RMA,STAT_DIV_NAME
0,3401744,1/07/2013,18.30.00,Yes,Struck Pedestrian,Monday,Dark Street lights on,Not at intersection,Serious injury accident,60 km/hr,261534,144.96980,-37.82202,Non-Intersection,MELBOURNE,METROPOLITAN NORTH WEST REGION,2497341.176,2408778.118,3,1,0,1,0,2,2,1,0,1,1,1,0,0,0,0,0,0,0,0,No,0,1,0,1,0,0,Local Road,Metro
1,3401745,2/07/2013,16.40.00,No,Collision with vehicle,Tuesday,Day,T intersection,Serious injury accident,60 km/hr,66573,145.14671,-37.83166,Intersection,WHITEHORSE,METROPOLITAN SOUTH EAST REGION,2512914.561,2407698.334,2,1,0,1,0,1,1,1,0,0,1,1,0,0,0,0,0,1,1,0,No,0,3,0,3,0,0,Arterial Other,Metro
2,3401746,2/07/2013,13.15.00,No,Collision with a fixed object,Tuesday,Day,T intersection,Serious injury accident,50 km/hr,261533,144.80134,-37.74003,Intersection,BRIMBANK,METROPOLITAN NORTH WEST REGION,2482490.641,2417859.686,1,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,Yes,0,1,0,1,0,0,Local Road,Metro
3,3401747,2/07/2013,16.45.00,No,Collision with a fixed object,Tuesday,Day,Not at intersection,Serious injury accident,110 km/hr,263495,145.07011,-37.17891,Non-Intersection,MITCHELL,NORTHERN REGION,2506225.129,2480145.341,3,2,0,2,0,1,1,2,0,2,1,0,0,0,0,0,0,0,0,0,No,0,1,0,1,0,0,Freeway,Country
4,3401748,2/07/2013,15.48.00,No,Collision with vehicle,Tuesday,Day,Not at intersection,Other injury accident,50 km/hr,261664,144.96530,-37.81808,Non-Intersection,MELBOURNE,METROPOLITAN NORTH WEST REGION,2496944.686,2409215.737,3,1,0,0,1,2,3,0,0,1,2,0,0,0,0,0,0,0,0,0,No,0,2,0,1,0,1,Local Road,Metro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65430,3467273,22/05/2018,15.44.00,No,Collision with vehicle,Tuesday,Day,Cross intersection,Other injury accident,60 km/hr,50677,142.47990,-38.37638,Intersection,WARRNAMBOOL,SOUTH WESTERN REGION,2279784.781,2344328.146,4,1,0,0,1,3,1,3,0,2,2,0,0,0,0,0,0,0,0,1,No,0,2,0,2,0,0,Arterial Highway,Country
65431,3467274,22/05/2018,17.32.00,No,Collision with vehicle,Tuesday,Dusk/Dawn,Not at intersection,Other injury accident,70 km/hr,331087,145.06291,-38.23802,Non-Intersection,MORNINGTON PENINSULA,METROPOLITAN SOUTH EAST REGION,2505508.040,2362601.303,4,1,0,0,1,3,1,3,0,1,3,0,0,0,0,0,0,0,0,0,No,0,3,0,3,0,0,Arterial Other,Metro
65432,3467275,22/05/2018,17.26.00,No,Collision with vehicle,Tuesday,Dark No street lights,Not at intersection,Other injury accident,60 km/hr,330794,145.33214,-38.09255,Non-Intersection,CASEY,METROPOLITAN SOUTH EAST REGION,2529136.402,2378700.466,2,1,0,0,1,1,2,0,0,0,2,0,0,0,0,0,0,0,1,0,No,0,2,1,1,0,0,Arterial Other,Metro
65433,3467276,13/12/2017,16.35.00,No,Vehicle overturned (no collision),Wednesday,Day,T intersection,Other injury accident,50 km/hr,332622,145.16357,-38.10343,Intersection,FRANKSTON,METROPOLITAN SOUTH EAST REGION,2514347.419,2377531.219,2,1,0,0,1,1,2,0,0,0,1,0,0,1,0,0,0,0,0,0,No,0,2,0,1,1,0,Local Road,Metro


In [17]:
#Delete the row with a MOUNT BULLER ALPINE RESOR Statistic
crash_demographics[crash_demographics.LGA_NAME != 'MOUNT BULLER ALPINE RESOR']

,OBJECTID,ACCIDENT_DATE,ACCIDENT_TIME,ALCOHOLTIME,ACCIDENT_TYPE,DAY_OF_WEEK,LIGHT_CONDITION,ROAD_GEOMETRY,SEVERITY,SPEED_ZONE,NODE_ID,LONGITUDE,LATITUDE,NODE_TYPE,LGA_NAME,REGION_NAME,VICGRID_X,VICGRID_Y,TOTAL_PERSONS,INJ_OR_FATAL,FATALITY,SERIOUSINJURY,OTHERINJURY,NONINJURED,MALES,FEMALES,BICYCLIST,PASSENGER,DRIVER,PEDESTRIAN,PILLION,MOTORIST,UNKNOWN,PED_CYCLIST_5_12,PED_CYCLIST_13_18,OLD_PEDESTRIAN,OLD_DRIVER,YOUNG_DRIVER,ALCOHOL_RELATED,UNLICENCSED,NO_OF_VEHICLES,HEAVYVEHICLE,PASSENGERVEHICLE,MOTORCYCLE,PUBLICVEHICLE,RMA,STAT_DIV_NAME
0,3401744,1/07/2013,18.30.00,Yes,Struck Pedestrian,Monday,Dark Street lights on,Not at intersection,Serious injury accident,60 km/hr,261534,144.96980,-37.82202,Non-Intersection,MELBOURNE,METROPOLITAN NORTH WEST REGION,2497341.176,2408778.118,3,1,0,1,0,2,2,1,0,1,1,1,0,0,0,0,0,0,0,0,No,0,1,0,1,0,0,Local Road,Metro
1,3401745,2/07/2013,16.40.00,No,Collision with vehicle,Tuesday,Day,T intersection,Serious injury accident,60 km/hr,66573,145.14671,-37.83166,Intersection,WHITEHORSE,METROPOLITAN SOUTH EAST REGION,2512914.561,2407698.334,2,1,0,1,0,1,1,1,0,0,1,1,0,0,0,0,0,1,1,0,No,0,3,0,3,0,0,Arterial Other,Metro
2,3401746,2/07/2013,13.15.00,No,Collision with a fixed object,Tuesday,Day,T intersection,Serious injury accident,50 km/hr,261533,144.80134,-37.74003,Intersection,BRIMBANK,METROPOLITAN NORTH WEST REGION,2482490.641,2417859.686,1,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,Yes,0,1,0,1,0,0,Local Road,Metro
3,3401747,2/07/2013,16.45.00,No,Collision with a fixed object,Tuesday,Day,Not at intersection,Serious injury accident,110 km/hr,263495,145.07011,-37.17891,Non-Intersection,MITCHELL,NORTHERN REGION,2506225.129,2480145.341,3,2,0,2,0,1,1,2,0,2,1,0,0,0,0,0,0,0,0,0,No,0,1,0,1,0,0,Freeway,Country
4,3401748,2/07/2013,15.48.00,No,Collision with vehicle,Tuesday,Day,Not at intersection,Other injury accident,50 km/hr,261664,144.96530,-37.81808,Non-Intersection,MELBOURNE,METROPOLITAN NORTH WEST REGION,2496944.686,2409215.737,3,1,0,0,1,2,3,0,0,1,2,0,0,0,0,0,0,0,0,0,No,0,2,0,1,0,1,Local Road,Metro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65430,3467273,22/05/2018,15.44.00,No,Collision with vehicle,Tuesday,Day,Cross intersection,Other injury accident,60 km/hr,50677,142.47990,-38.37638,Intersection,WARRNAMBOOL,SOUTH WESTERN REGION,2279784.781,2344328.146,4,1,0,0,1,3,1,3,0,2,2,0,0,0,0,0,0,0,0,1,No,0,2,0,2,0,0,Arterial Highway,Country
65431,3467274,22/05/2018,17.32.00,No,Collision with vehicle,Tuesday,Dusk/Dawn,Not at intersection,Other injury accident,70 km/hr,331087,145.06291,-38.23802,Non-Intersection,MORNINGTON PENINSULA,METROPOLITAN SOUTH EAST REGION,2505508.040,2362601.303,4,1,0,0,1,3,1,3,0,1,3,0,0,0,0,0,0,0,0,0,No,0,3,0,3,0,0,Arterial Other,Metro
65432,3467275,22/05/2018,17.26.00,No,Collision with vehicle,Tuesday,Dark No street lights,Not at intersection,Other injury accident,60 km/hr,330794,145.33214,-38.09255,Non-Intersection,CASEY,METROPOLITAN SOUTH EAST REGION,2529136.402,2378700.466,2,1,0,0,1,1,2,0,0,0,2,0,0,0,0,0,0,0,1,0,No,0,2,1,1,0,0,Arterial Other,Metro
65433,3467276,13/12/2017,16.35.00,No,Vehicle overturned (no collision),Wednesday,Day,T intersection,Other injury accident,50 km/hr,332622,145.16357,-38.10343,Intersection,FRANKSTON,METROPOLITAN SOUTH EAST REGION,2514347.419,2377531.219,2,1,0,0,1,1,2,0,0,0,1,0,0,1,0,0,0,0,0,0,No,0,2,0,1,1,0,Local Road,Metro


### Final tidy up of the data to get ready for analysis

In [18]:
#Create a new DataFrame to enable work below (cleaned)
crash_demographic_clean = crash_demographics[["OBJECTID", "LGA_NAME", "TOTAL_PERSONS", "INJ_OR_FATAL", "FATALITY", "SERIOUSINJURY", "OTHERINJURY", "NONINJURED", "ALCOHOL_RELATED", "MALES", "FEMALES", "OLD_DRIVER", "YOUNG_DRIVER"]]


In [19]:
#Group data to calculate the number of crashes per LGA (excluding alcohol related crashes)
LGA_crash_count = crash_demographic_clean.groupby(["LGA_NAME"])["OBJECTID"].count()
LGA_total_inj = crash_demographic_clean.groupby(["LGA_NAME"])["TOTAL_PERSONS"].sum()
LGA_total_inj_or_fatal = crash_demographic_clean.groupby(["LGA_NAME"])["INJ_OR_FATAL"].sum()
LGA_total_fatal = crash_demographic_clean.groupby(["LGA_NAME"])["FATALITY"].sum()
LGA_total_serious = crash_demographic_clean.groupby(["LGA_NAME"])["SERIOUSINJURY"].sum()
LGA_total_minor = crash_demographic_clean.groupby(["LGA_NAME"])["OTHERINJURY"].sum()
LGA_total_no_injury = crash_demographic_clean.groupby(["LGA_NAME"])["NONINJURED"].sum()
LGA_total_male = crash_demographic_clean.groupby(["LGA_NAME"])["MALES"].sum()
LGA_total_female = crash_demographic_clean.groupby(["LGA_NAME"])["FEMALES"].sum()
LGA_total_old = crash_demographic_clean.groupby(["LGA_NAME"])["OLD_DRIVER"].sum()
LGA_total_young = crash_demographic_clean.groupby(["LGA_NAME"])["YOUNG_DRIVER"].sum()

In [20]:
#Calculate the number of alcohol related crashes and group by lGA 
LGA_count_alcohol = crash_demographic_clean.loc[crash_demographic_clean["ALCOHOL_RELATED"] == "Yes"]
LGA_total_alcohol = LGA_count_alcohol.groupby(["LGA_NAME"])["ALCOHOL_RELATED"].count()

In [21]:
#Create a new grouped dataframe 
grouped_LGA_totals = pd.DataFrame({"total_crashes": LGA_crash_count,
                                  "total_inj": LGA_total_inj,
                                  "inj_fatal": LGA_total_inj_or_fatal,
                                   "fatal": LGA_total_fatal,
                                   "serious": LGA_total_serious,
                                   "minor": LGA_total_minor,
                                   "no_inj": LGA_total_no_injury,
                                   "alcohol": LGA_total_alcohol,
                                   "male": LGA_total_male,
                                   "female": LGA_total_female,
                                   "old_driver": LGA_total_old,
                                   "young_driver": LGA_total_young
                                  })

In [22]:
#Replace the NaN in the file with a 0
grouped_LGA_totals["alcohol"] = grouped_LGA_totals["alcohol"].fillna(0)

In [23]:
#Set the index
group_LGA = grouped_LGA_totals.rename_axis("LGA").reset_index()

In [24]:
group_LGA.head(50)

,LGA,total_crashes,total_inj,inj_fatal,fatal,serious,minor,no_inj,alcohol,male,female,old_driver,young_driver
0,ALPINE,227,437,281,4,93,184,156,6.0,274,147,19,27
1,ARARAT,150,344,221,8,71,142,123,5.0,207,132,13,35
2,BALLARAT,1134,2918,1531,20,316,1195,1387,56.0,1412,1345,101,406
3,BANYULE,909,2224,1127,10,247,870,1097,18.0,1158,973,81,289
4,BASS COAST,340,878,485,11,142,332,393,20.0,470,386,32,95
5,BAW BAW,780,1404,950,16,292,642,454,28.0,848,515,37,208
6,BAYSIDE,799,1882,964,12,290,662,918,20.0,1028,793,107,177
7,BENALLA,192,484,249,10,96,143,235,5.0,267,200,21,52
8,BENDIGO,1282,2941,1610,23,426,1161,1331,53.0,1522,1307,104,432
9,BOROONDARA,1440,3312,1630,14,384,1232,1682,37.0,1791,1368,117,358


In [25]:
#Merge the Database to do the demographics analysis
LGA_merged = pd.merge(lga_demo_df2, group_LGA, how="outer", on="LGA")

In [26]:
LGA_merged

,LGA,LGA Code,Departmental Region,City_Regional,Index of Relative Socio-Economic Disadvantage (IRSD),Index of Relative Socio-Economic Disadvantage (IRSD) (rank),Percentage of highly disadvantaged SA1s,Percentage of highly disadvantaged SA1s (rank),People with income less than $400 per week,People with income less than $400 per week (rank),Median household income,Median household income (rank),Median house price,Median house price (rank),Journeys to work which are by car,Journeys to work which are by car (rank),People with at least 2 hour daily commute,People with at least 2 hour daily commute (rank),People at increased risk of alcohol-related harm on a single occasion of drinking,People at increased risk of alcohol-related harm on a single occasion of drinking (rank),People reporting fair or poor health status,People reporting fair or poor health status (rank),"Clients that received Alcohol & Drug Treatment Services per 1,000 population","Clients that received Alcohol & Drug Treatment Services per 1,000 population (rank)",Total_pop,total_crashes,total_inj,inj_fatal,fatal,serious,minor,no_inj,alcohol,male,female,old_driver,young_driver
0,ALPINE,20110,Hume,Regional,987,35,0.238,33,0.448,21,829,67,265000,50,0.439,77,0.083,35,0.561,6,0.193,10,1.0,78,12335,227,437,281,4,93,184,156,6.0,274,147,19,27
1,ARARAT,20260,Grampians,Regional,951,11,0.455,8,0.433,30,844,64,193000,68,0.687,38,0.000,0,0.482,31,0.144,45,11.5,4,11599,150,344,221,8,71,142,123,5.0,207,132,13,35
2,BALLARAT,20570,Grampians,Regional,981,29,0.315,23,0.417,39,988,44,294000,48,0.738,15,0.047,47,0.537,15,0.123,60,5.5,31,101689,1134,2918,1531,20,316,1195,1387,56.0,1412,1345,101,406
3,BANYULE,20660,Northern Metropolitan,Metro,1047,67,0.083,60,0.369,68,1394,12,620000,17,0.641,55,0.206,5,0.407,64,0.167,31,3.7,54,121869,909,2224,1127,10,247,870,1097,18.0,1158,973,81,289
4,BASS COAST,20740,Gippsland,Regional,978,27,0.216,36,0.438,26,855,61,340000,38,0.714,25,0.068,40,0.452,44,0.177,20,7.1,17,32806,340,878,485,11,142,332,393,20.0,470,386,32,95
5,BAW BAW,20830,Gippsland,Regional,998,47,0.152,54,0.415,41,1025,40,309000,44,0.699,32,0.108,25,0.429,59,0.129,58,5.1,35,48477,780,1404,950,16,292,642,454,28.0,848,515,37,208
6,BAYSIDE,20910,Southern Metropolitan,Metro,1091,77,0.023,73,0.327,76,1826,3,1250000,3,0.627,57,0.141,20,0.505,23,0.085,78,2.8,67,97092,799,1882,964,12,290,662,918,20.0,1028,793,107,177
7,BENALLA,21010,Hume,Regional,957,16,0.297,26,0.448,22,827,69,235500,59,0.693,34,0.000,0,0.402,66,0.098,71,3.2,62,13863,192,484,249,10,96,143,235,5.0,267,200,21,52
8,BENDIGO,22620,Loddon Mallee,Regional,983,31,0.280,28,0.413,43,991,43,320000,41,0.740,13,0.000,0,0.441,50,0.172,26,7.1,15,110479,1282,2941,1610,23,426,1161,1331,53.0,1522,1307,104,432
9,BOROONDARA,21110,Eastern Metropolitan,Metro,1098,78,0.008,75,0.333,75,1893,1,1550000,1,0.566,68,0.055,43,0.488,28,0.103,68,2.5,70,167232,1440,3312,1630,14,384,1232,1682,37.0,1791,1368,117,358


In [27]:
#Add in additional column to represent the total crashes per person in Victoria
LGA_merged["crash_per_person"] = LGA_merged["total_crashes"] / LGA_merged["Total_pop"]

In [28]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
LGA_merged.to_csv("lgamerged.csv", encoding="utf-8", index=False)

## Work out overall statistics - by age and gender classifications (Young, Older, Middle Aged)

In [29]:
#Group data to calculate the number of crashes per region (excluding alcohol related crashes)
region_crash_count2 = LGA_merged.groupby(["Departmental Region", "City_Regional"])["total_crashes"].sum()
region_total_injuries = LGA_merged.groupby(["Departmental Region", "City_Regional"])["total_inj"].sum()
region_total_fatal = LGA_merged.groupby(["Departmental Region", "City_Regional"])["fatal"].sum()
region_total_male = LGA_merged.groupby(["Departmental Region", "City_Regional"])["male"].sum()
region_total_female = LGA_merged.groupby(["Departmental Region", "City_Regional"])["female"].sum()
region_total_old = LGA_merged.groupby(["Departmental Region", "City_Regional"])["old_driver"].sum()
region_total_young = LGA_merged.groupby(["Departmental Region", "City_Regional"])["young_driver"].sum()


In [30]:
#Create a new grouped dataframe 
grouped_LGA_demos = pd.DataFrame({"total_crashes2": region_crash_count2,
                                  "total_inj": region_total_injuries,
                                  "fatal": region_total_fatal,
                                  "inj_male": region_total_male,
                                   "inj_female": region_total_female,
                                   "old_driver": region_total_old,
                                   "young_driver": region_total_young,
                                   })

In [31]:
#Have a look at the condensed df
grouped_LGA_demos

,,total_crashes2,total_inj,fatal,inj_male,inj_female,old_driver,young_driver
Departmental Region,City_Regional,,,,,,,
Barwon-South Western,Regional,4299,10328,130,5529,4341,340,1411
Eastern Metropolitan,Metro,9491,23029,122,12269,9729,765,3108
Gippsland,Regional,3532,7169,121,4167,2794,234,959
Grampians,Regional,2883,6407,114,3533,2629,206,878
Hume,Regional,4016,8182,185,4897,2983,248,990
Loddon Mallee,Regional,3631,7936,130,4416,3263,303,1065
Northern Metropolitan,Metro,10097,24854,93,13681,9749,513,3102
Southern Metropolitan,Metro,14361,35660,194,19285,14516,1056,4395
Western Metropolitan,Metro,10727,26969,138,15289,10075,401,3075


In [32]:
#Calculate some additional ratios 
#Total injuries per crash
grouped_LGA_demos["inj_per_crash"] = grouped_LGA_demos["total_inj"] / grouped_LGA_demos["total_crashes2"]

#% Male injuries
grouped_LGA_demos["male_%"] = grouped_LGA_demos["inj_male"] / grouped_LGA_demos["total_inj"]

#% Female injuries
grouped_LGA_demos["female_%"] = grouped_LGA_demos["inj_female"] / grouped_LGA_demos["total_inj"]

#% Total Crashes involving an older person driver
grouped_LGA_demos["older_%"] = grouped_LGA_demos["old_driver"] / grouped_LGA_demos["total_crashes2"]

#% Total Crashes involving an older person driver
grouped_LGA_demos["young_%"] = grouped_LGA_demos["young_driver"] / grouped_LGA_demos["total_crashes2"]


In [33]:
grouped_LGA_demos

,,total_crashes2,total_inj,fatal,inj_male,inj_female,old_driver,young_driver,inj_per_crash,male_%,female_%,older_%,young_%
Departmental Region,City_Regional,,,,,,,,,,,,
Barwon-South Western,Regional,4299,10328,130,5529,4341,340,1411,2.402419,0.535341,0.420314,0.079088,0.328216
Eastern Metropolitan,Metro,9491,23029,122,12269,9729,765,3108,2.426404,0.532763,0.422467,0.080603,0.327468
Gippsland,Regional,3532,7169,121,4167,2794,234,959,2.029728,0.581253,0.389734,0.066251,0.271518
Grampians,Regional,2883,6407,114,3533,2629,206,878,2.222338,0.551428,0.410332,0.071453,0.304544
Hume,Regional,4016,8182,185,4897,2983,248,990,2.037351,0.598509,0.364581,0.061753,0.246514
Loddon Mallee,Regional,3631,7936,130,4416,3263,303,1065,2.185624,0.556452,0.411164,0.083448,0.293308
Northern Metropolitan,Metro,10097,24854,93,13681,9749,513,3102,2.461523,0.550455,0.392251,0.050807,0.307220
Southern Metropolitan,Metro,14361,35660,194,19285,14516,1056,4395,2.483114,0.540802,0.407067,0.073532,0.306037
Western Metropolitan,Metro,10727,26969,138,15289,10075,401,3075,2.514123,0.566910,0.373577,0.037382,0.286660


In [34]:
#Sort by young driver %
region_plot = grouped_LGA_demos.copy()
region_plot.reset_index()

,Departmental Region,City_Regional,total_crashes2,total_inj,fatal,inj_male,inj_female,old_driver,young_driver,inj_per_crash,male_%,female_%,older_%,young_%
0,Barwon-South Western,Regional,4299,10328,130,5529,4341,340,1411,2.402419,0.535341,0.420314,0.079088,0.328216
1,Eastern Metropolitan,Metro,9491,23029,122,12269,9729,765,3108,2.426404,0.532763,0.422467,0.080603,0.327468
2,Gippsland,Regional,3532,7169,121,4167,2794,234,959,2.029728,0.581253,0.389734,0.066251,0.271518
3,Grampians,Regional,2883,6407,114,3533,2629,206,878,2.222338,0.551428,0.410332,0.071453,0.304544
4,Hume,Regional,4016,8182,185,4897,2983,248,990,2.037351,0.598509,0.364581,0.061753,0.246514
5,Loddon Mallee,Regional,3631,7936,130,4416,3263,303,1065,2.185624,0.556452,0.411164,0.083448,0.293308
6,Northern Metropolitan,Metro,10097,24854,93,13681,9749,513,3102,2.461523,0.550455,0.392251,0.050807,0.307220
7,Southern Metropolitan,Metro,14361,35660,194,19285,14516,1056,4395,2.483114,0.540802,0.407067,0.073532,0.306037
8,Western Metropolitan,Metro,10727,26969,138,15289,10075,401,3075,2.514123,0.566910,0.373577,0.037382,0.286660


In [35]:
#Group data to calculate the number of crashes per region (excluding alcohol related crashes)
region_crash_count3 = LGA_merged.groupby(["City_Regional"])["total_crashes"].sum()
region_total_injuries3 = LGA_merged.groupby(["City_Regional"])["total_inj"].sum()
region_total_fatal3 = LGA_merged.groupby(["City_Regional"])["fatal"].sum()
region_total_male3 = LGA_merged.groupby(["City_Regional"])["male"].sum()
region_total_female3 = LGA_merged.groupby(["City_Regional"])["female"].sum()
region_total_old3 = LGA_merged.groupby(["City_Regional"])["old_driver"].sum()
region_total_young3 = LGA_merged.groupby(["City_Regional"])["young_driver"].sum()
region_total_pop3 = LGA_merged.groupby(["City_Regional"])["Total_pop"].sum()

In [36]:
#Create a new grouped dataframe 
grouped_city_or_regional = pd.DataFrame({"total_crashes": region_crash_count3,
                                  "total_inj": region_total_injuries3,
                                  "fatal": region_total_fatal3,       
                                  "inj_male": region_total_male3,
                                   "inj_female": region_total_female3,
                                   "old_driver": region_total_old3,
                                   "young_driver": region_total_young3,
                                   "total_pop": region_total_pop3
})

In [37]:
grouped_city_or_regional

,total_crashes,total_inj,fatal,inj_male,inj_female,old_driver,young_driver,total_pop
City_Regional,,,,,,,,
Metro,44676,110512,547,60524,44069,2735,13680,4415403
Regional,18361,40022,680,22542,16010,1331,5303,1502734


In [38]:
#Calculate some additional ratios 
#Total injuries per crash
grouped_city_or_regional["inj_per_crash"] = grouped_city_or_regional["total_inj"] / grouped_city_or_regional["total_crashes"]

#% Male injuries
grouped_city_or_regional["male_%"] = grouped_city_or_regional["inj_male"] / grouped_city_or_regional["total_inj"]

#% Female injuries
grouped_city_or_regional["female_%"] = grouped_city_or_regional["inj_female"] / grouped_city_or_regional["total_inj"]

#% Total Crashes involving an older person driver
grouped_city_or_regional["older_%"] = grouped_city_or_regional["old_driver"] / grouped_city_or_regional["total_crashes"]

#% Total Crashes involving an older person driver
grouped_city_or_regional["young_%"] = grouped_city_or_regional["young_driver"] / grouped_city_or_regional["total_crashes"]

#split of fatalities to metro / regional
grouped_city_or_regional["percent_fatal"] = grouped_city_or_regional["fatal"] / (grouped_city_or_regional["fatal"].sum())


In [39]:
#Calculate the Metro / Regional split of accidents for high level analysis
grouped_city_or_regional["percent_crashes"] = grouped_city_or_regional["total_crashes"] / (grouped_city_or_regional["total_crashes"].sum())

In [40]:
#Calculate the Metro / Regional split of population for high level analysis
grouped_city_or_regional["percent_pop"] = grouped_city_or_regional["total_pop"] / (grouped_city_or_regional["total_pop"].sum())

In [41]:
grouped_city_or_regional

,total_crashes,total_inj,fatal,inj_male,inj_female,old_driver,young_driver,total_pop,inj_per_crash,male_%,female_%,older_%,young_%,percent_fatal,percent_crashes,percent_pop
City_Regional,,,,,,,,,,,,,,,,
Metro,44676,110512,547,60524,44069,2735,13680,4415403,2.473632,0.547669,0.398771,0.061219,0.306205,0.445803,0.708727,0.74608
Regional,18361,40022,680,22542,16010,1331,5303,1502734,2.179729,0.563240,0.400030,0.072491,0.288819,0.554197,0.291273,0.25392
